In [1]:
import pandas as pd
import sys
sys.path.append('./utils')
from utils import *
from rank import *
from process import * 
import tqdm

In [2]:
# データ保存先
data_folda = '../data/'

* masterからデータを追加

In [3]:
# make_timeseries_data.ipynb で作成したデータを読み込む
pseudo_time_series=pd.read_pickle(data_folda+'skeleton_train.pkl')
# base_date が 2023年1月1日以降のデータを抽出 (closed_date がそれ以降の閉店店舗しかクロールしていないため)
pseudo_time_series=pseudo_time_series[pseudo_time_series['base_date']>'2023-01-01']
pseudo_time_series = add_master_data(pseudo_time_series, data_folda, "restaurant_data.csv")
pseudo_time_series.to_pickle(data_folda+'train_with_master.pkl')

pseudo_time_series_test=pd.read_pickle(data_folda+'skeleton_test.pkl')
pseudo_time_series_test = add_master_data(pseudo_time_series_test, data_folda, "restaurant_data.csv")
pseudo_time_series_test.to_pickle(data_folda+'test_with_master.pkl')

del pseudo_time_series, pseudo_time_series_test

* V_GOOGLE からデータを追加

In [4]:
pseudo_time_series = pd.read_pickle(data_folda+'train_with_master.pkl')
add_list=["restaurant_id","overview.aggregateRating","overview.reviewCount","general_rank"]
pseudo_time_series = add_v_google(pseudo_time_series, data_folda, add_list)
pseudo_time_series.to_pickle(data_folda+'train_with_master_google.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'test_with_master.pkl')
add_list=["restaurant_id","overview.aggregateRating","overview.reviewCount","general_rank"]
pseudo_time_series_test = add_v_google(pseudo_time_series_test, data_folda, add_list)
pseudo_time_series_test.to_pickle(data_folda+'test_with_master_google.pkl')

del pseudo_time_series, pseudo_time_series_test

/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

* V_RETTY からデータを追加

In [5]:
#追加する変数をリストで定義
add_list=["restaurant_id",'infos.isOfficial', 'infos.familiar.users.genge',
       'infos.familiar.users.count', 'infos.familiar.stars',
       'general_rank_RETTY', 'infos.reserveEntireStore',
       'infos.onlineReservation', 'infos.access.transferTime1',
       'infos.numberOfSeats.value', 'infos.counterSeats.value',
       'infos.privateRoom.value', 'infos.updateInfo.firstReviewDate',
       'infos.wantToGo', 'infos.updateInfo.lastReviewDate',
       'infos.went', 'infos.rateByWent', 'infos.excellent', 'infos.coupon',
       'infos.photoCount.Photograph', 'infos.photoCount.Cooking',
       'infos.photoCount.Interior', 'infos.photoCount.Exterior',
       'infos.photoCount.Menu', 'infos.reviewCount.Review',
       'infos.reviewCount.Lunch', 'infos.reviewCount.Dinner',
       'menus.course_list_count',
       'menus.dishes_list_count', 'menus.lunch_list_count',
       'menus.drink_list_count', 'menus.takeout_list_count']
 
pseudo_time_series = pd.read_pickle(data_folda+'train_with_master_google.pkl')         
pseudo_time_series = add_v_retty(pseudo_time_series, data_folda, add_list, master_name="V_RETTY",read_from_pickle=False)
pseudo_time_series.to_pickle(data_folda+'train_with_master_google_retty.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'test_with_master_google.pkl')
pseudo_time_series_test = add_v_retty(pseudo_time_series_test, data_folda, add_list, master_name="V_RETTY",read_from_pickle=False)
pseudo_time_series_test.to_pickle(data_folda+'test_with_master_google_retty.pkl')

del pseudo_time_series, pseudo_time_series_test

/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

* HOTPEPPER からデータを追加

In [6]:
add_list=['restaurant_id','infos.aggregateRating','infos.ratingReview','infos.satisfaction.percentage1']

pseudo_time_series = pd.read_pickle(data_folda+'train_with_master_google_retty.pkl')
pseudo_time_series = add_v_hotpepper(pseudo_time_series, data_folda, add_list, master_name="V_HOTPEPPER",read_from_pickle=False)
pseudo_time_series.to_pickle(data_folda+'train_with_master_google_retty_hotpepper.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'test_with_master_google_retty.pkl')
pseudo_time_series_test = add_v_hotpepper(pseudo_time_series_test, data_folda, add_list, master_name="V_HOTPEPPER",read_from_pickle=False) 
pseudo_time_series_test.to_pickle(data_folda+'test_with_master_google_retty_hotpepper.pkl')

del pseudo_time_series, pseudo_time_series_test

/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

* V_STRUCTURED からデータを取得

In [3]:
add_list = '"restaurant_id","url","name_rank","address_rank","telephone_rank","aggregateRating.ratingCount","aggregateRating.ratingValue", "aggregateRating.reviewCount", \
    "aggregateRating.bestRating", "aggregateRating.worstRating", "price", \
    "priceRange", "servesCuisine","interactionCount.went", \
    "interactionCount.wanttogo","paymentAccepted", \
    "acceptsReservations", \
    "ReserveAction.result.name", "OrderAction.target.actionPlatform", \
    "OrderAction.target.inLanguage", "OrderAction.target.urlTemplate"'


pseudo_time_series = pd.read_pickle(data_folda+'train_with_master_google_retty_hotpepper.pkl')
pseudo_time_series = add_v_structured(pseudo_time_series, data_folda, add_list, master_name="V_HOTPEPPER",read_from_pickle=False)
pseudo_time_series.to_pickle(data_folda+'train_with_master_google_retty_hotpepper_structured.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'test_with_master_google_retty_hotpepper.pkl')
pseudo_time_series_test = add_v_structured(pseudo_time_series_test, data_folda, add_list, master_name="V_HOTPEPPER",read_from_pickle=False) 
pseudo_time_series_test.to_pickle(data_folda+'test_with_master_google_retty_hotpepper_structured.pkl')

del pseudo_time_series, pseudo_time_series_test